In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time
import os

In [ ]:
OUTDIR = 'workspace/01-reviews'
os.makedirs(OUTDIR, exist_ok=True)

In [3]:
def get_soup(item_id, page):
    url = f"https://review.rakuten.co.jp/item/1/{item_id}?p={page}#itemReviewList"
    
    # ページの内容を取得
    response = requests.get(url)
    response.encoding = 'utf-8'  # エンコーディングを指定
    
    # BeautifulSoupでHTMLを解析
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [4]:
def get_reviews_from_a_page(soup):
    # レビューのリストを格納するためのリスト
    reviews = []

    # レビュー要素を全て取得
    review_elements = soup.find_all('div', class_='review-body--1pESv')

    for element in review_elements:
        # レビュー本文を取得
        review_text = element.get_text(strip=True)
        reviews.append(review_text)

    return reviews

In [ ]:
def get_all_review(item_id):
    soup = get_soup(item_id, 1)

    # 最大page取得
    max_page = int(soup.find_all('button', {'class': "page-button--1Lwus"})[-1].get_text())

    # 1ページ目
    print(f"1 / {max_page}")
    reviews = get_reviews_from_a_page(soup)

    # 2ページ目以降
    for page in range(2, max_page+1):
        time.sleep(10)
        print(f"{page} / {max_page}")
        soup = get_soup(item_id, page)
        reviews.extend(get_reviews_from_a_page(soup))

    return reviews

# start to get data

In [6]:
item_ids = ["411011_10000344"]

In [ ]:
for item_id in item_ids:
    reviews = get_all_review(item_id)
    with open(f"{OUTDIR}/{item_id}.json", "w", encoding="utf-8") as f:
        json.dump(reviews, f, ensure_ascii=False, indent=2)

1 / 85
